In [1]:
#import os
#os.system('GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"')

In [3]:
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import vaex

import glob
import itertools as itr
import collections as coll

import time

import sqlalchemy as sqla
import types
import json

In [13]:
%%capture
from tqdm.auto import trange, tqdm
tqdm.pandas()

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Replace the first with the second category
CATEGORY_ALIASES = {
    'math.MP': 'math-ph',
    'stat.TH': 'math.ST',
    'math.IT': 'cs.IT',
    'q-fin.EC': 'econ.GN',
    'cs.SY': 'eess.SY',
    'cs.NA': 'math.NA'
}

# Used subject categories, with aliases removed.
ACTIVE_CATEGORIES = [
    'astro-ph.CO','astro-ph.EP','astro-ph.GA',
    'astro-ph.HE','astro-ph.IM','astro-ph.SR',
    'cond-mat.dis-nn','cond-mat.mes-hall','cond-mat.mtrl-sci',
    'cond-mat.other','cond-mat.quant-gas','cond-mat.soft',
    'cond-mat.stat-mech','cond-mat.str-el','cond-mat.supr-con',
    'cs.AI','cs.AR','cs.CC','cs.CE','cs.CG','cs.CL','cs.CR',
    'cs.CV','cs.CY','cs.DB','cs.DC','cs.DL','cs.DM','cs.DS',
    'cs.ET','cs.FL','cs.GL','cs.GR','cs.GT','cs.HC','cs.IR',
    'cs.IT','cs.LG','cs.LO','cs.MA','cs.MM','cs.MS','cs.NE',
    'cs.NI','cs.OH','cs.OS','cs.PF','cs.PL','cs.RO','cs.SC',
    'cs.SD','cs.SE','cs.SI',
    'econ.EM','econ.GN','econ.TH',
    'eess.AS','eess.IV','eess.SP','eess.SY',
    'gr-qc',
    'hep-ex','hep-lat','hep-ph','hep-th',
    'math-ph','math.AC','math.AG','math.AP',
    'math.AT','math.CA','math.CO','math.CT',
    'math.CV','math.DG','math.DS','math.FA',
    'math.GM','math.GN','math.GR','math.GT',
    'math.HO','math.KT','math.LO','math.MG',
    'math.NA','math.NT','math.OA','math.OC',
    'math.PR','math.QA','math.RA','math.RT',
    'math.SG','math.SP','math.ST',
    'nlin.AO','nlin.CD','nlin.CG','nlin.PS','nlin.SI',
    'nucl-ex','nucl-th',
    'physics.acc-ph','physics.ao-ph','physics.app-ph',
    'physics.atm-clus','physics.atom-ph','physics.bio-ph',
    'physics.chem-ph','physics.class-ph','physics.comp-ph',
    'physics.data-an','physics.ed-ph','physics.flu-dyn',
    'physics.gen-ph','physics.geo-ph','physics.hist-ph',
    'physics.ins-det','physics.med-ph','physics.optics',
    'physics.plasm-ph','physics.pop-ph','physics.soc-ph',
    'physics.space-ph',
    'q-bio.BM','q-bio.CB','q-bio.GN','q-bio.MN','q-bio.NC',
    'q-bio.OT','q-bio.PE','q-bio.QM','q-bio.SC','q-bio.TO',
    'q-fin.CP','q-fin.GN','q-fin.MF','q-fin.PM',
    'q-fin.PR','q-fin.RM','q-fin.ST','q-fin.TR',
    'quant-ph',
    'stat.AP','stat.CO','stat.ME','stat.ML','stat.OT',
    'dg-ga', 'astro-ph'
]

CAT_REMAPPER = {x:x for x in ACTIVE_CATEGORIES} | CATEGORY_ALIASES


len(CATEGORY_ALIASES)
len(ACTIVE_CATEGORIES)
len(CAT_REMAPPER)

6

151

157

In [5]:
train_df = vaex.open("./data/fulltext/train_minor_cats.hdf5")
test_df = vaex.open("./data/fulltext/test_minor_cats.hdf5")

In [6]:
train_df.shape

(112026, 9)

In [7]:
from google.cloud import storage as gs
from google.cloud import exceptions as gerr

In [8]:
key_file_path = "/home/cjc73/.credentials/arxiv-production-e9574603e738.json"
client = gs.Client.from_service_account_json(key_file_path)

def get_last_version(paper):
    prefix_path = '/'.join([
        'txt',
        paper['major_category'] if '/' in paper['paper_id'] else 'arxiv', 
        paper['yymm'],
        paper['paper_id'].split('/')[-1],
        #"1001",
        #paper['yymm'],
        #paper['']
    ])
    blobs = client.list_blobs(
        'arxiv-production-data', 
        prefix=prefix_path
    )
    return max(blobs, key=lambda x: x.name) #latest version

Need to extract the components from record to build a path and test with different permissions. 

    

In [9]:
train_df.shape[0]

112026

In [11]:
#paper = dict(zip(train_df.column_names, train_df[0]))
#get_last_version(paper)

<Blob: arxiv-production-data, txt/arxiv/1501/1501.03486v2.txt, 1628901958760505>

In [ ]:
with open('data/fulltext/train_minor_cats_full.json', 'w') as outfile, \
    tqdm(total=train_df.shape[0]) as err_bar:
    
    fail_count = 0
    fail_idx = []
    decode_fail_count = 0
    decode_fail_idx = []

    for i, row in tqdm(enumerate(train_df.iterrows()), total=train_df.shape[0]):
        if i==0: print(row)
        paper = row[1]
        try:
            blob = get_last_version(paper)
            textbytes = blob.download_as_bytes()
        except (ValueError, gerr.GoogleCloudError):
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
            continue
            
        except gerr.GoogleCloudError:
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
        try: 
            text_string = textbytes.decode('utf-8')
            paper['fulltext'] = text_string  
        except UnicodeDecodeError:
            try:
                text_string = textbytes.decode("windows-1252")
                paper['fulltext'] = text_string  
            except:
                try:
                    text_string = textbytes.decode("cp1251")
                    paper['fulltext'] = text_string  
                except:
                    decode_fail_count += 1
                    decode_fail_idx.append(row[0])
                    _ = err_bar.update(err_bar.n + 1)

        paper['created'] = paper['created'].isoformat()
        _ = outfile.write(
                json.dumps(
                    paper, 
                    ensure_ascii=False
                )
            )
        _ = outfile.write('\n')
        time.sleep(.3)

print(f"{fail_count} errors of {i+1} completed.")
print(f"{decode_fail_count} decode fails of {i+1} completed.")
    
        
with open('data/fulltext/train_minor_fail_idx.txt', 'w') as outfile:
    for x in fail_idx:
        outfile.write(f"{x}\n")         


In [ ]:
with open('data/fulltext/test_minor_cats_full.json', 'w') as outfile, \
    tqdm(total=test_df.shape[0]) as err_bar:
    
    fail_count = 0
    fail_idx = []
    decode_fail_count = 0
    decode_fail_idx = []

    for i, row in tqdm(enumerate(test_df.iterrows()), total=test_df.shape[0]):
        #print(row)
        paper = row[1]
        try:
            blob = get_last_version(paper)
            textbytes = blob.download_as_bytes()
        except (ValueError, gerr.GoogleCloudError):
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
            continue
            
        except gerr.GoogleCloudError:
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
        try: 
            text_string = textbytes.decode('utf-8')
            paper['fulltext'] = text_string  
        except UnicodeDecodeError:
            try:
                text_string = textbytes.decode("windows-1252")
                paper['fulltext'] = text_string  
            except:
                try:
                    text_string = textbytes.decode("cp1251")
                    paper['fulltext'] = text_string  
                except:
                    decode_fail_count += 1
                    decode_fail_idx.append(row[0])
                    _ = err_bar.update(err_bar.n + 1)

        paper['created'] = paper['created'].isoformat()
        _ = outfile.write(
                json.dumps(
                    paper, 
                    ensure_ascii=False
                )
            )
        _ = outfile.write('\n')
        time.sleep(.3)

print(f"{fail_count} errors of {i+1} completed.")
print(f"{decode_fail_count} decode fails of {i+1} completed.")
    
with open('data/fulltext/test_minor_fail_idx.txt', 'w') as outfile:
    for x in fail_idx:
        outfile.write(f"{x}\n")         
        

  0%|          | 0/7938 [00:00<?, ?it/s]

  0%|          | 0/7938 [00:00<?, ?it/s]